In [ ]:
import numpy as np
import pandas as pd

import os
from os.path import join, abspath, dirname, realpath
import sys

import datetime
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta
from dateutil import relativedelta
from itertools import islice

import matplotlib.pyplot as plt
import seaborn as sns

import csv
import sklearn
import sklearn.preprocessing
from datetime import date, timedelta
import dateutil
from dateutil.relativedelta import relativedelta
from functions import *

In [ ]:
df_clean=pd.read_csv("df_clean.csv")
weather_full=pd.read_csv("weather_full.csv")

#correcting the type for every variable
df_clean['temp_avg']=df_clean['temp_avg'].astype(float)
df_clean['date']=pd.to_datetime(df_clean['date'])
df_clean.reset_index(drop=True, inplace=True)
df_clean['migraine_start']=df_clean['migraine_start'].astype(int)

weather_full['date']=pd.to_datetime(weather_full['date'])


In [ ]:
print("last date:",max(df_clean['date']))
#Data split in train and test set. Avoidance of autocorrelation by NOT using same time periods and same patients 
train, test = data_split(df_clean,test_first_day=datetime.datetime(2021, 1, 16),test_size=relativedelta(years=1))
print("test/train size:",round(test.shape[0]/train.shape[0],2))
print("missing data",round(1-(train.shape[0]+test.shape[0])/231117,2))
print('-----') 


In [ ]:
#DATA_PATH = abspath(join(os.getcwd(),"Desktop","df_clean_plus.csv"))

train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

train.to_csv("train.csv",index=False)
test.to_csv("test.csv",index=False)

## train_plus.csv

train data with the addition of imputation and order of sequences

In [ ]:
#setting a new variable for the order to the patients' sequences, so that they are processed individually
train_clean=order_split(train)
#imputation variable indicates whether data entry is imputed or not
train_clean['imputation']=0
missin=fill_missing(train_clean,weather_full)
missin.reset_index(drop=True, inplace=True)
#adding imputed data
train_plus = train_clean.append(missin)
train_plus['migraine_start']=train_plus['migraine_start'].astype(int)
train_plus['imputation']=train_plus['imputation'].astype(int)
train_plus['date']=pd.to_datetime(train_plus['date'])
train_plus.to_csv("train_plus.csv",index=False)

## test_plus.csv


test data with the addition of imputation and order of sequences

In [ ]:
#setting a new variable for the order to the patients' sequences, so that they are processed individually
test_clean=order_split(test)
#imputation variable indicates whether data entry is imputed or not
test_clean['imputation']=0
missin=fill_missing(test_clean,weather_full)
missin.reset_index(drop=True, inplace=True)
#adding imputed data
test_plus = test_clean.append(missin)
test_plus['migraine_start']=test_plus['migraine_start'].astype(int)
test_plus['imputation']=test_plus['imputation'].astype(int)
test_plus['date']=pd.to_datetime(test_plus['date'])
test_plus.to_csv("test_plus.csv",index=False)

## df_clean_plus.csv

same process for the creation of the full dataset + imputation + order

In [ ]:
df_clean_plusx=order_split(df_clean)
df_clean_plusx['imputation']=0
missin=fill_missing(df_clean_plusx,weather_full)
missin.reset_index(drop=True, inplace=True)
df_clean_plus = df_clean_plusx.append(missin)

df_clean_plus['migraine_start']=df_clean_plus['migraine_start'].astype(int)
df_clean_plus['imputation']=df_clean_plus['imputation'].astype(int)
df_clean_plus['date']=pd.to_datetime(df_clean_plus['date'])

df_clean_plus.to_csv('df_clean_plus.csv',index=False)

## Days per patient

In [ ]:
df=df_clean
d=[]
#Calculation of the days per patient
for i in np.unique(df['patient_id']):
    total=(max(df.loc[df.patient_id==i]['date'])-min(df.loc[df.patient_id==i]['date'])).days
    d.append(sum(df.loc[df.patient_id==i]['migraine_start'])*2+total)


In [ ]:
#Total number of days in train set
max(train['date'])-min(train['date'])#.days/6


## Split for cross validation

In [ ]:
s=pd.date_range(start=min(train['date']),end=max(train['date']))
periods=np.array_split(s, 5)
periods

# Data split graphs

In [ ]:
patients=[]
#patient's data for the duration of 12 months in the train set
for i in (range(12)):
    x=train[train['date'].dt.month==i+1]
    y=x['date'].dt.day.value_counts()
    y=y.sort_index()
    y=y.to_list()
    patients.extend(y)

#same for the test set
patients_test=[]
for i in (range(12)):
    x=test[test['date'].dt.month==i+1]
    y=x['date'].dt.day.value_counts()
    y=y.sort_index()
    y=y.to_list()
    patients_test.extend(y)


In [ ]:
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(10)
plt.ylim([0, 550])

plt.rcParams.update({'font.size': 15})

plt.plot(range(366),patients,color='g',label='train')
plt.plot(range(365),patients_test,color='r',label='test')

plt.legend()
plt.xlabel("Days")
plt.title("Number of patients per day")
plt.ylabel("Patients")
plt.show()

In [ ]:
total_days=max(df['date'])-min(df['date'])
pat_num=[]
first_day=min(df['date'])

#finding number of patients per day for every day of the study
for i in range(total_days.days+1):
    
    day = first_day + timedelta(days=i)
    pat_num.append(df[df['date']==day].shape[0])


In [ ]:
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(10)
plt.rcParams.update({'font.size': 15})

plt.plot(range(total_days.days+1),pat_num)
plt.axvline(x=total_days.days,color='r')
plt.axvline(x=total_days.days-365,color='r')
plt.axvline(x=total_days.days-730,color='r')
plt.axvline(x=total_days.days-1095,color='r')

plt.xlabel("Days")

plt.ylabel("Patients")
plt.show()